In [ ]:
import pandas as pd
import numpy as np

Task 1: Data Preprocessing

In [ ]:
anime_df = pd.read_csv('/content/anime.csv')

In [ ]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
anime_df.tail()

,anime_id,name,genre,type,episodes,rating,members
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175
12293,26081,Yasuji no Pornorama: Yacchimae!!,Hentai,Movie,1,5.46,142


In [ ]:
anime_df.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [ ]:
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [ ]:
anime_df.isnull().sum()

,0
anime_id,0
name,0
genre,62
type,25
episodes,0
rating,230
members,0


In [ ]:
anime_df.shape

(12294, 7)

Task 2: Feature Extraction

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler

In [ ]:
# Fill missing values
anime_df['genre'] = anime_df['genre'].fillna('')
anime_df['rating'] = anime_df['rating'].fillna(anime_df['rating'].mean())

In [ ]:
# One-hot encode the genres
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(anime_df['genre'].str.split(','))

In [ ]:
# Normalize the average rating
scaler = MinMaxScaler()
ratings_normalized = scaler.fit_transform(anime_df[['rating']])


In [ ]:
# Combine the features into a single DataFrame
features_df = pd.DataFrame(genres_encoded, columns=mlb.classes_)
features_df['rating'] = ratings_normalized

In [ ]:
print(features_df.head())

       Adventure   Cars   Comedy   Dementia   Demons   Drama   Ecchi  \
0  0           0      0        0          0        0       0       0   
1  0           1      0        0          0        0       1       0   
2  0           0      0        1          0        0       0       0   
3  0           0      0        0          0        0       0       0   
4  0           0      0        1          0        0       0       0   

    Fantasy   Game  ...  Shounen  Slice of Life  Space  Sports  Super Power  \
0         0      0  ...        0              0      0       0            0   
1         1      0  ...        0              0      0       0            0   
2         0      0  ...        0              0      0       0            0   
3         0      0  ...        0              0      0       0            0   
4         0      0  ...        0              0      0       0            0   

   Supernatural  Thriller  Vampire  Yaoi    rating  
0             0         0        0     

Task 3: Recommendation System


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine, correlation

In [ ]:
# User similarity - pairwise - taking 2 rows at a time it will compute distance
# That distance is scaled between 0 to 1
# Suppose distance between 2 rows is 0.  We want similarity, not a distance.
# That is calculated as Similarity=1-distance. So 1-0=1. i.e. similarity in rows is 1.
# Suppose distance is 0.9. So 1-0.9=0.1
user_sim = 1 - pairwise_distances(features_df.values,metric='cosine')

In [ ]:
user_sim

array([[1.        , 0.13661081, 0.13644987, ..., 0.15085865, 0.15492584,
        0.1737458 ],
       [0.13661081, 1.        , 0.36135915, ..., 0.11708593, 0.12024259,
        0.13484933],
       [0.13644987, 0.36135915, 1.        , ..., 0.116948  , 0.12010094,
        0.13469047],
       ...,
       [0.15085865, 0.11708593, 0.116948  , ..., 1.        , 0.99994581,
        0.99824985],
       [0.15492584, 0.12024259, 0.12010094, ..., 0.99994581, 1.        ,
        0.99881138],
       [0.1737458 , 0.13484933, 0.13469047, ..., 0.99824985, 0.99881138,
        1.        ]])

In [ ]:
np.fill_diagonal(user_sim, 0)

In [ ]:
user_sim

array([[0.        , 0.13661081, 0.13644987, ..., 0.15085865, 0.15492584,
        0.1737458 ],
       [0.13661081, 0.        , 0.36135915, ..., 0.11708593, 0.12024259,
        0.13484933],
       [0.13644987, 0.36135915, 0.        , ..., 0.116948  , 0.12010094,
        0.13469047],
       ...,
       [0.15085865, 0.11708593, 0.116948  , ..., 0.        , 0.99994581,
        0.99824985],
       [0.15492584, 0.12024259, 0.12010094, ..., 0.99994581, 0.        ,
        0.99881138],
       [0.1737458 , 0.13484933, 0.13469047, ..., 0.99824985, 0.99881138,
        0.        ]])

In [ ]:
#Store the results in a dataframe
user_sim_df = pd.DataFrame(user_sim)

In [ ]:
user_sim_df

,0,1,2,3,4,5,6,7,8,9,...,12284,12285,12286,12287,12288,12289,12290,12291,12292,12293
0,0.000000,0.136611,0.136450,0.225314,0.134998,0.344700,0.171458,0.378285,0.134028,0.134190,...,0.132814,0.381158,0.142157,0.127622,0.129360,0.119713,0.125440,0.150859,0.154926,0.173746
1,0.136611,0.000000,0.361359,0.174873,0.360552,0.417958,0.622314,0.295929,0.360010,0.360101,...,0.103081,0.102865,0.110333,0.099052,0.100400,0.092913,0.097358,0.117086,0.120243,0.134849
2,0.136450,0.361359,0.000000,0.174667,0.999993,0.269528,0.459122,0.295795,0.999981,0.999984,...,0.102960,0.102744,0.110203,0.098935,0.100282,0.092803,0.097243,0.116948,0.120101,0.134690
3,0.225314,0.174873,0.174667,0.000000,0.172809,0.200133,0.219480,0.218989,0.171566,0.171774,...,0.170013,0.169657,0.181973,0.163367,0.165592,0.153242,0.160573,0.193111,0.198318,0.222409
4,0.134998,0.360552,0.999993,0.172809,0.000000,0.268424,0.458116,0.294589,0.999997,0.999998,...,0.101864,0.101651,0.109030,0.097882,0.099215,0.091816,0.096209,0.115704,0.118823,0.133258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0.119713,0.092913,0.092803,0.153242,0.091816,0.106334,0.116613,0.116352,0.091156,0.091266,...,0.999464,0.090141,0.998414,0.999805,0.999710,0.000000,0.999898,0.996924,0.996054,0.990544
12290,0.125440,0.097358,0.097243,0.160573,0.096209,0.111421,0.122192,0.121919,0.095517,0.095632,...,0.999829,0.094454,0.999116,0.999985,0.999952,0.999898,0.000000,0.997941,0.997219,0.992402
12291,0.150859,0.117086,0.116948,0.193111,0.115704,0.133999,0.146952,0.146624,0.114872,0.115011,...,0.998956,0.113594,0.999755,0.998276,0.998522,0.996924,0.997941,0.000000,0.999946,0.998250
12292,0.154926,0.120243,0.120101,0.198318,0.118823,0.137611,0.150914,0.150577,0.117969,0.118112,...,0.998426,0.116656,0.999470,0.997611,0.997902,0.996054,0.997219,0.999946,0.000000,0.998811


In [ ]:
#Set the index and column names to user ids
user_sim_df.index = anime_df.	anime_id.unique()
user_sim_df.columns = anime_df.	anime_id.unique()

In [ ]:
user_sim_df

,32281,5114,28977,9253,9969,32935,11061,820,15335,15417,...,26031,34399,10368,9352,5541,9316,5543,5621,6133,26081
32281,0.000000,0.136611,0.136450,0.225314,0.134998,0.344700,0.171458,0.378285,0.134028,0.134190,...,0.132814,0.381158,0.142157,0.127622,0.129360,0.119713,0.125440,0.150859,0.154926,0.173746
5114,0.136611,0.000000,0.361359,0.174873,0.360552,0.417958,0.622314,0.295929,0.360010,0.360101,...,0.103081,0.102865,0.110333,0.099052,0.100400,0.092913,0.097358,0.117086,0.120243,0.134849
28977,0.136450,0.361359,0.000000,0.174667,0.999993,0.269528,0.459122,0.295795,0.999981,0.999984,...,0.102960,0.102744,0.110203,0.098935,0.100282,0.092803,0.097243,0.116948,0.120101,0.134690
9253,0.225314,0.174873,0.174667,0.000000,0.172809,0.200133,0.219480,0.218989,0.171566,0.171774,...,0.170013,0.169657,0.181973,0.163367,0.165592,0.153242,0.160573,0.193111,0.198318,0.222409
9969,0.134998,0.360552,0.999993,0.172809,0.000000,0.268424,0.458116,0.294589,0.999997,0.999998,...,0.101864,0.101651,0.109030,0.097882,0.099215,0.091816,0.096209,0.115704,0.118823,0.133258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9316,0.119713,0.092913,0.092803,0.153242,0.091816,0.106334,0.116613,0.116352,0.091156,0.091266,...,0.999464,0.090141,0.998414,0.999805,0.999710,0.000000,0.999898,0.996924,0.996054,0.990544
5543,0.125440,0.097358,0.097243,0.160573,0.096209,0.111421,0.122192,0.121919,0.095517,0.095632,...,0.999829,0.094454,0.999116,0.999985,0.999952,0.999898,0.000000,0.997941,0.997219,0.992402
5621,0.150859,0.117086,0.116948,0.193111,0.115704,0.133999,0.146952,0.146624,0.114872,0.115011,...,0.998956,0.113594,0.999755,0.998276,0.998522,0.996924,0.997941,0.000000,0.999946,0.998250
6133,0.154926,0.120243,0.120101,0.198318,0.118823,0.137611,0.150914,0.150577,0.117969,0.118112,...,0.998426,0.116656,0.999470,0.997611,0.997902,0.996054,0.997219,0.999946,0.000000,0.998811


In [ ]:
# idxmax() method returns a Series with the index of the maximum value for each column. (row 3 anc col 11 has highest value as 1)
# By specifying the column axis (axis='columns' or 1), the idxmax() method returns a Series with the index of the maximum value for each row.
user_sim_df.idxmax(axis=1)

,0
32281,547
5114,121
28977,9969
9253,11577
9969,15417
32935,28891
11061,136
820,3665
15335,15417
15417,15335


In [ ]:
#Most Similar Users
user_sim_df.idxmax(axis=1)[0:10]

,0
32281,547
5114,121
28977,9969
9253,11577
9969,15417
32935,28891
11061,136
820,3665
15335,15417
15417,15335


In [ ]:
user_sim_df.iloc[0:5, 0:5]
# similarity 1 - between customer 3 to 3, 6 to 6.
# similarity between customer 11 and customer 3 is 1. i.e. they are very similar.

,32281,5114,28977,9253,9969
32281,0.000000,0.136611,0.136450,0.225314,0.134998
5114,0.136611,0.000000,0.361359,0.174873,0.360552
28977,0.136450,0.361359,0.000000,0.174667,0.999993
9253,0.225314,0.174873,0.174667,0.000000,0.172809
9969,0.134998,0.360552,0.999993,0.172809,0.000000


In [ ]:
# eg. find movies watched by customer 6 and 168 as they are similar
anime_df[(anime_df['anime_id']==6) | (anime_df['anime_id']==168)]
# Both watched Toy Story with good rating, 6 watched 2 more movies.
# Now rating for Sabrina is more than other movie. So we can recommend that movie to 168.

,anime_id,name,genre,type,episodes,rating,members
214,6,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069
1761,168,s.CRY.ed,"Action, Adventure, Drama, Sci-Fi, Super Power",TV,26,7.45,52784


Task 4: Evaluation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
# Split the dataset
train_df, test_df = train_test_split(anime_df, test_size=0.2, random_state=42)

In [ ]:
# Example of evaluation
# This is a placeholder as the evaluation method depends on user-item interactions
# Normally, you'd compare the recommendations to user preferences

In [ ]:
# Calculate precision, recall, and F1-score
# For simplicity, using a dummy implementation
precision = precision_score([1, 0, 1], [1, 0, 1])
recall = recall_score([1, 0, 1], [1, 0, 1])
f1 = f1_score([1, 0, 1], [1, 0, 1])

In [ ]:
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


Conclusion

The implementation of the anime recommendation system using cosine similarity demonstrates a foundational approach by preprocessing data, extracting, encoding, and normalizing key features like genre and ratings. The recommendation function suggests anime based on cosine similarity.

Interview Questions:
1. Can you explain the difference between user-based and item-based collaborative filtering?

Ans:-

Collaborative Filtering (CF) is a technique used in recommendation systems to predict a user’s preferences based on the preferences of other users. It relies on user interactions with items (e.g., ratings, likes) to make recommendations.

1. User-Based Collaborative Filtering:

Concept: This method recommends items by finding users similar to the target user and suggesting items those similar users liked.

How It Works:
Identify users who have similar preferences to the target user based on historical data (e.g., ratings).
Recommend items that these similar users liked or rated highly.

Example: If User A and User B have similar movie ratings, and User A liked a specific movie that User B hasn’t seen yet, that movie would be recommended to User B.

2. Item-Based Collaborative Filtering:

Concept: This method recommends items based on the similarity between items rather than users.

How It Works:
Calculate the similarity between items based on user interactions (e.g., users who liked Item X also liked Item Y).
Recommend items that are similar to items the target user has liked or interacted with.

Example: If a user likes a particular movie, the system finds other movies similar to that one and recommends those to the user.

2. What is collaborative filtering, and how does it work?

Ans:-

Collaborative Filtering (CF) is a technique used in recommendation systems to suggest items (e.g., movies, products) based on the preferences of users with similar tastes. It leverages the collective behavior of users to make recommendations.

How It Works:

Data Collection: Collect user-item interaction data (e.g., ratings, purchase history).
Similarity Computation:
User-Based: Compute similarity between users based on their interactions or ratings.
Item-Based: Compute similarity between items based on how users have interacted with them.

Recommendation Generation:
User-Based: Recommend items liked by similar users.
Item-Based: Recommend items similar to those the user has interacted with.
Prediction: Generate recommendations based on the computed similarities and user interactions.

Example: If you frequently rate action movies highly, the system may recommend new action movies or similar genres based on the preferences of other users who also rate action movies highly.